In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D, Flatten
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [2]:
classifier = Sequential()

# convolution layer - 32 feature detectors
classifier.add(Conv2D(32, 3, input_shape=(64, 64, 3), activation='relu'))

# max pooling layer - of 2x2 shape
classifier.add(MaxPool2D(pool_size=(2, 2)))

# another convolution layer
classifier.add(Conv2D(32, 3, activation='relu'))

# another max pooling layer
classifier.add(MaxPool2D(pool_size=(2, 2)))

# flattening layer
classifier.add(Flatten())

# fully connected layer
classifier.add(Dense(units=128, activation='relu'))

# output layer
classifier.add(Dense(units=4, activation='softmax'))

In [3]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory("C:/Users/vivek jha/Documents/CNN_assignment/Dataset/train",
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical'
                                                )

validation_set = test_datagen.flow_from_directory("drive/My Drive/CNN assignment/Dataset/Validate",
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode='categorical')

In [7]:
from keras.callbacks import ModelCheckpoint

filepath = "C:/Users/vivek jha/Documents/CNN_assignment/Dataset/saved_model/weights-improvement-{epoch:02d}-{loss: .4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch=360,                         
                         epochs=50,                                 
                         validation_data=validation_set,
                         validation_steps=80,
                         callbacks=callbacks_list)                             

In [8]:
#Loading Epoch 9/50
#360/360 [==============================] - 190s 528ms/step - loss: 1.2399e-05 - accuracy: 1.0000 - val_loss: 1.7182e-04 - val_accuracy: 0.9751

classifier.load_weights("C:/Users/vivek jha/Documents/CNN_assignment/Dataset/saved_model/weights-improvement-09- 0.0000.hdf5")

In [9]:
folder_path =  "C:/Users/vivek jha/Documents/CNN_assignment/Dataset/test"

images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
images[0].shape

(1, 64, 64, 3)

In [14]:
image=[]
predicted=[]
for img in range(30):
    classes=classifier.predict_classes(images[img])
    image.append(os.listdir(folder_path))
    predicted.append(classes+1)
    dict={"image_id":image[img],"predicted_labels":predicted}
data=pd.DataFrame(dict)
data.to_csv("result.csv",index=False)